In [9]:
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import emcee
import corner

measuredNuclearData = np.loadtxt("ZNBE.dat")

for i in measuredNuclearData:
    
    i[2] = i[2] * (i[0] + i[1])

In [10]:
def sampleSurface(inputVars, measuredNuclearData):
    
    chiSq = 0
    SEMFData = []
    
    Av, As, Ac, Aa, Ap = inputVars
    
    for i in measuredNuclearData:
        
        Z = i[0]
        N = i[1]
        A = N + Z

        vol = Av * A
        sur = As * A**(2/3)
        cou = Ac * (Z * (Z - 1)) * A**(-1/3)
        asy = Aa * (N - Z)**2 * A**(-1)

        if Z%2==0 and N%2==0:
            pai = Ap * A**-0.5
        elif Z%2!=0 and N%2!=0:
            pai = -Ap * A**-0.5
        else:
            pai = 0

        BE = vol + sur + cou + asy + pai

        SEMFData.append([A, BE])
        chiSq += (BE - i[2])**2

    return -round(chiSq, 3)

In [11]:
def prior_knowledge(p0, limits):
    
    if p0[0] > limits[0][0] or p0[0] < limits[0][1]:
        return -np.inf
    
    if p0[1] > limits[1][0] or p0[0] < limits[1][1]:
        return -np.inf
    
    if p0[2] > limits[2][0] or p0[0] < limits[2][1]:
        return -np.inf
    
    if p0[3] > limits[3][0] or p0[0] < limits[3][1]:
        return -np.inf
    
    if p0[4] > limits[4][0] or p0[0] < limits[4][1]:
        return -np.inf
    
    return 0.0

def prob_fn(p0, measuredNuclearData, limits):
    
    lp = prior_knowledge(p0, limits)
    
    if not np.isfinite(lp):
        return -np.inf
    
    return lp + sampleSurface(p0, measuredNuclearData)

In [ ]:
nwalkers = 100
ndim = 5

limits = [[], [], [], [], []]

limits[0] = [50, 0]
limits[1] = [0, -50]
limits[2] = [0, -50]
limits[3] = [0, -50]
limits[4] = [0, -50]

original_limits = limits

fig = plt.figure(figsize=(8,8))

for i in range(1000):

    AvField = np.linspace(limits[0][1], limits[0][0], 100)
    AsField = np.linspace(limits[1][1], limits[1][0], 100)
    AcField = np.linspace(limits[2][1], limits[2][0], 100)
    AaField = np.linspace(limits[3][1], limits[3][0], 100)
    ApField = np.linspace(limits[4][1], limits[4][0], 100)

    p0 = []
    
    for i in range(nwalkers):
        Av = round(AvField[np.random.randint(0, 100)], 1)
        As = round(AsField[np.random.randint(0, 100)], 1)
        Ac = round(AcField[np.random.randint(0, 100)], 1)
        Aa = round(AaField[np.random.randint(0, 100)], 1)
        Ap = round(ApField[np.random.randint(0, 100)], 1)

        p0.append([Av, As, Ac, Aa, Ap])

    sampler = emcee.EnsembleSampler(nwalkers, ndim, prob_fn, args=[measuredNuclearData, limits])

    start = time.perf_counter()
    sampler.run_mcmc(p0, 100);
    stop = time.perf_counter()

    print("Done")
    stop = time.perf_counter()
    print("Time taken: ", stop - start)

    flat_samples = sampler.get_chain(flat=True)
    print(flat_samples.shape)
    labels = ["Av", "As", "Ac", "Aa", "Ap"]
    corner.corner(flat_samples, labels=labels, quantiles=[0.16, 0.5, 0.84], show_titles=True, figure=fig)
    fig.clf()
    fig.show()

    limits = [[], [], [], [], []]

    limits[0] = corner.quantile(flat_samples[:, 0], [0.95, 0.05])
    limits[1] = corner.quantile(flat_samples[:, 1], [0.95, 0.05])
    limits[2] = corner.quantile(flat_samples[:, 2], [0.95, 0.05])
    limits[3] = corner.quantile(flat_samples[:, 3], [0.95, 0.05])
    limits[4] = corner.quantile(flat_samples[:, 4], [0.95, 0.05])

    for i in range(len(original_limits)):

        if limits[i][0] > original_limits[i][0]:
            limits[i][0] = original_limits[i][0]
        if limits[i][1] < original_limits[i][1]:
            limits[i][1] = original_limits[i][1]

Done
Time taken:  110.78308874999993
(10000, 5)


/var/folders/gf/62k0bxjn45dc8x13pb588v980000gn/T/ipykernel_52397/83395644.py:50: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
/Users/willgraham/opt/anaconda3/lib/python3.8/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]
